In [ ]:
#@title Default title text
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [1]:
import pandas as pd
import numpy as np

def to_month_id(year, month):
  return (year - 1996) * 12 + month - 6

n_features = 12
lookback = 3
chosen_stocks = ["AAPL"] # "AMZN", "MSFT", "IBM", "INTC", "QCOM", "NVDA", "IBM", "ADBE", "EBAY", "CSCO"

In [2]:
dfFeature = pd.read_csv("./lstm_2004_12.csv")
# dfFeature.loc[dfFeature["return"] == 0, "return"] = 1
dfFeature = dfFeature[dfFeature.QAId.isin(chosen_stocks)]
features = dfFeature.drop(columns=['month_id', 'QAId']).as_matrix()
print(features.shape)

(157, 12)


In [3]:
from sklearn.preprocessing import MinMaxScaler
normalizedFeatures = MinMaxScaler().fit_transform(features) \
                                   .reshape(157, len(chosen_stocks), n_features) \
                                   .reshape(157, len(chosen_stocks) * n_features)
print(normalizedFeatures.shape)

(157, 12)


In [4]:
dfTarget = pd.read_csv("./return_2004_40.csv")
dfTarget["return"] = np.sign(dfTarget["return"])
# dfTarget.loc[dfTarget["return"] == 0, "return"] = 1
dfTarget = dfTarget[dfTarget.QAId.isin(chosen_stocks)]
dfTarget = dfTarget[dfTarget.month_id >= (to_month_id(2004, 1) + lookback)]
targets = MinMaxScaler().fit_transform(dfTarget.drop(columns=['month_id', 'QAId']).as_matrix())
y = targets.reshape(157 - lookback + 1, len(chosen_stocks))
print(y.shape)

(155, 1)


In [5]:
from keras.preprocessing.sequence import TimeseriesGenerator

splittedFeature = normalizedFeatures
print(splittedFeature.shape)

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(157, 12)


In [6]:
from keras.preprocessing.sequence import TimeseriesGenerator

x = np.zeros((157 - lookback + 1, lookback, n_features * len(chosen_stocks)))
y_mock = np.zeros((157, len(chosen_stocks)))

i = 0
for train, test in TimeseriesGenerator(splittedFeature, y_mock, length=lookback, batch_size=1):
  if i > 157 - lookback:
    break
  x[i] = train[0]
  i = i + 1

In [7]:
from sklearn.model_selection import PredefinedSplit

test_months = 26
end_point = 157 - lookback + 1
split_point = 157 - lookback + 1 - test_months

x_train = x[0:split_point - 48].reshape(split_point - 48, lookback, n_features * len(chosen_stocks))
y_train = y[0:split_point - 48].reshape(split_point - 48, len(chosen_stocks))
x_validate = x[split_point - 36:split_point].reshape(36, lookback, n_features * len(chosen_stocks))
y_validate = y[split_point - 36:split_point].reshape(36, len(chosen_stocks))
x_test = x[split_point:end_point].reshape(test_months, lookback, n_features * len(chosen_stocks))
y_test = y[split_point:end_point].reshape(test_months, len(chosen_stocks))

print(x_train.shape)
print(y_train.shape)
print(x_validate.shape)
print(y_validate.shape)
print(x_test.shape)
print(y_test.shape)

# ps = PredefinedSplit(np.append(np.negative(np.ones(60 * 465)), np.zeros(24 * 465))).split(x_train)

# for train_ids, test_ids in ps:
#   print(str(train_ids) + ", " + str(test_ids))

(81, 3, 12)
(81, 1)
(36, 3, 12)
(36, 1)
(26, 3, 12)
(26, 1)


In [8]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Flatten, CuDNNLSTM
from keras.regularizers import l1_l2, l2
from keras.optimizers import RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

np.random.seed(4103)

def create_model(*param):
    model = Sequential()
    model.add(LSTM(units=256, input_shape=(lookback, n_features * len(chosen_stocks)), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=128, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(len(chosen_stocks), activation="relu"))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001, decay=0.0), metrics=['accuracy'], *param)
    return model

In [9]:
batch_size=30

model = create_model()
train_score = model.fit(x_train, y_train, validation_data=(x_validate, y_validate), batch_size=batch_size, epochs=200)
print(train_score)

Train on 81 samples, validate on 36 samples
Epoch 1/200
81/81 [==============================] - 2s 25ms/step - loss: 8.7425 - acc: 0.3210 - val_loss: 3.7995 - val_acc: 0.3333
Epoch 2/200
81/81 [==============================] - 0s 724us/step - loss: 4.6263 - acc: 0.3210 - val_loss: 3.2512 - val_acc: 0.3333
Epoch 3/200
81/81 [==============================] - 0s 739us/step - loss: 3.2837 - acc: 0.3210 - val_loss: 2.9922 - val_acc: 0.3333
Epoch 4/200
81/81 [==============================] - 0s 744us/step - loss: 3.0240 - acc: 0.3210 - val_loss: 2.8321 - val_acc: 0.3333
Epoch 5/200
81/81 [==============================] - 0s 745us/step - loss: 2.8663 - acc: 0.3210 - val_loss: 2.7140 - val_acc: 0.3333
Epoch 6/200
81/81 [==============================] - 0s 743us/step - loss: 2.7318 - acc: 0.3210 - val_loss: 2.6181 - val_acc: 0.3333
Epoch 7/200
81/81 [==============================] - 0s 829us/step - loss: 2.6505 - acc: 0.3210 - val_loss: 2.5367 - val_acc: 0.3333
Epoch 8/200
81/81 [=======

81/81 [==============================] - 0s 799us/step - loss: 0.8029 - acc: 0.2840 - val_loss: 0.7817 - val_acc: 0.3333
Epoch 63/200
81/81 [==============================] - 0s 822us/step - loss: 0.7762 - acc: 0.2963 - val_loss: 0.7625 - val_acc: 0.3333
Epoch 64/200
81/81 [==============================] - 0s 800us/step - loss: 0.7620 - acc: 0.3333 - val_loss: 0.7446 - val_acc: 0.3611
Epoch 65/200
81/81 [==============================] - 0s 798us/step - loss: 0.7378 - acc: 0.4321 - val_loss: 0.7287 - val_acc: 0.3333
Epoch 66/200
81/81 [==============================] - 0s 745us/step - loss: 0.7204 - acc: 0.4074 - val_loss: 0.7142 - val_acc: 0.4444
Epoch 67/200
81/81 [==============================] - 0s 833us/step - loss: 0.7300 - acc: 0.3580 - val_loss: 0.7015 - val_acc: 0.5000
Epoch 68/200
81/81 [==============================] - 0s 757us/step - loss: 0.6996 - acc: 0.5432 - val_loss: 0.6909 - val_acc: 0.6111
Epoch 69/200
81/81 [==============================] - 0s 730us/step - loss:

81/81 [==============================] - 0s 781us/step - loss: 0.6736 - acc: 0.6420 - val_loss: 0.6547 - val_acc: 0.6389
Epoch 124/200
81/81 [==============================] - 0s 787us/step - loss: 0.6752 - acc: 0.5926 - val_loss: 0.6542 - val_acc: 0.6389
Epoch 125/200
81/81 [==============================] - 0s 748us/step - loss: 0.6591 - acc: 0.6914 - val_loss: 0.6538 - val_acc: 0.6389
Epoch 126/200
81/81 [==============================] - 0s 794us/step - loss: 0.6543 - acc: 0.6420 - val_loss: 0.6533 - val_acc: 0.6389
Epoch 127/200
81/81 [==============================] - 0s 811us/step - loss: 0.6416 - acc: 0.6667 - val_loss: 0.6523 - val_acc: 0.6389
Epoch 128/200
81/81 [==============================] - 0s 758us/step - loss: 0.6783 - acc: 0.6667 - val_loss: 0.6519 - val_acc: 0.6389
Epoch 129/200
81/81 [==============================] - 0s 761us/step - loss: 0.6335 - acc: 0.6790 - val_loss: 0.6516 - val_acc: 0.6389
Epoch 130/200
81/81 [==============================] - 0s 742us/step 

Epoch 184/200
81/81 [==============================] - 0s 888us/step - loss: 0.6548 - acc: 0.6049 - val_loss: 0.6487 - val_acc: 0.6667
Epoch 185/200
81/81 [==============================] - 0s 903us/step - loss: 0.6565 - acc: 0.6790 - val_loss: 0.6490 - val_acc: 0.6667
Epoch 186/200
81/81 [==============================] - 0s 853us/step - loss: 0.6639 - acc: 0.6667 - val_loss: 0.6492 - val_acc: 0.6667
Epoch 187/200
81/81 [==============================] - 0s 888us/step - loss: 0.6247 - acc: 0.6914 - val_loss: 0.6493 - val_acc: 0.6667
Epoch 188/200
81/81 [==============================] - 0s 822us/step - loss: 0.6532 - acc: 0.6667 - val_loss: 0.6489 - val_acc: 0.6667
Epoch 189/200
81/81 [==============================] - 0s 818us/step - loss: 0.6409 - acc: 0.6667 - val_loss: 0.6486 - val_acc: 0.6667
Epoch 190/200
81/81 [==============================] - 0s 830us/step - loss: 0.6464 - acc: 0.6790 - val_loss: 0.6486 - val_acc: 0.6667
Epoch 191/200
81/81 [==============================] - 

In [10]:
import matplotlib.pyplot as plt

test_score = model.evaluate(x_test, y_test, batch_size=batch_size)
for i, mn in enumerate(model.metrics_names):
  print(mn + ": " + str(test_score[i]))

plt.plot(train_score.history["loss"])
plt.plot(train_score.history["val_loss"])
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Metrics")
plt.legend(["Training", "Validation"], loc="upper left")
plt.show()

26/26 [==============================] - 0s 183us/step
loss: 0.6704335808753967
acc: 0.5384615659713745


In [11]:
from sklearn.metrics import precision_score, recall_score, classification_report

def threshold_tester(result, target, a, b):
  for t in np.arange(a, b + 0.05, 0.05):
    print("thresholder: " + str(t))
    print(classification_report(target, (result >= t).astype(int)))

In [12]:
result = model.predict(x_test, batch_size=batch_size)
print(result)
threshold_tester(result, y_test, 0.1, 0.9)

[[0.613702  ]
 [0.59174865]
 [0.6047674 ]
 [0.63382846]
 [0.5526133 ]
 [0.57229304]
 [0.5803692 ]
 [0.56547344]
 [0.5257256 ]
 [0.51530886]
 [0.6128672 ]
 [0.58287615]
 [0.5800873 ]
 [0.5469766 ]
 [0.52951425]
 [0.5928227 ]
 [0.5981025 ]
 [0.5679567 ]
 [0.56059194]
 [0.6085009 ]
 [0.6169292 ]
 [0.60256624]
 [0.5749369 ]
 [0.5992197 ]
 [0.55680346]
 [0.831459  ]]
thresholder: 0.1
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        12
        1.0       0.54      1.00      0.70        14

avg / total       0.29      0.54      0.38        26

thresholder: 0.15000000000000002
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        12
        1.0       0.54      1.00      0.70        14

avg / total       0.29      0.54      0.38        26

thresholder: 0.20000000000000004
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        12
        1.0    

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
import math

buy_list = []
sell_list = []

midpt = (max(map(lambda x: x[0], result)) + min(map(lambda x: x[0], result))) / 2

for i, r in enumerate(result):
  buy_or_sell = 1 if r.item() > midpt * 1 else (-1 if r.item() < midpt * 0.9 else 0)
  if r.item() > midpt * 1.1:
    buy_list.append({'month_id': i + 223, 'QAId': chosen_stocks[0]})
  if r.item() < midpt * 0.9:
    sell_list.append({'month_id': i + 223, 'QAId': chosen_stocks[0]})
  print(str(y_test[i].item()) + ": " + str(buy_or_sell))
  if (math.fabs(buy_or_sell - y_test[i].item()) == 2) or (buy_or_sell - y_test[i].item() == 1):
    print("Hey")
print(buy_list)
print(sell_list)

1.0: 0
0.0: -1
1.0: -1
Hey
1.0: 0
0.0: -1
0.0: -1
0.0: -1
0.0: -1
1.0: -1
Hey
0.0: -1
0.0: 0
0.0: -1
0.0: -1
1.0: -1
Hey
0.0: -1
1.0: -1
Hey
0.0: -1
1.0: -1
Hey
1.0: -1
Hey
1.0: 0
1.0: 0
0.0: -1
1.0: -1
Hey
1.0: -1
Hey
1.0: -1
Hey
1.0: 1
[{'month_id': 248, 'QAId': 'AAPL'}]
[{'month_id': 224, 'QAId': 'AAPL'}, {'month_id': 225, 'QAId': 'AAPL'}, {'month_id': 227, 'QAId': 'AAPL'}, {'month_id': 228, 'QAId': 'AAPL'}, {'month_id': 229, 'QAId': 'AAPL'}, {'month_id': 230, 'QAId': 'AAPL'}, {'month_id': 231, 'QAId': 'AAPL'}, {'month_id': 232, 'QAId': 'AAPL'}, {'month_id': 234, 'QAId': 'AAPL'}, {'month_id': 235, 'QAId': 'AAPL'}, {'month_id': 236, 'QAId': 'AAPL'}, {'month_id': 237, 'QAId': 'AAPL'}, {'month_id': 238, 'QAId': 'AAPL'}, {'month_id': 239, 'QAId': 'AAPL'}, {'month_id': 240, 'QAId': 'AAPL'}, {'month_id': 241, 'QAId': 'AAPL'}, {'month_id': 244, 'QAId': 'AAPL'}, {'month_id': 245, 'QAId': 'AAPL'}, {'month_id': 246, 'QAId': 'AAPL'}, {'month_id': 247, 'QAId': 'AAPL'}]


In [18]:
dfBuy = pd.DataFrame(buy_list, columns=["month_id", "QAId"])
dfSell = pd.DataFrame(sell_list, columns=["month_id", "QAId"])
dfBuy.head(5)
dfSell.head(5)

,month_id,QAId
0,224,AAPL
1,225,AAPL
2,227,AAPL
3,228,AAPL
4,229,AAPL


In [19]:
# from google.colab import files

filename_base = "_".join([stock.lower() for stock in chosen_stocks])
# filename_model = "./" + filename_base + "_model.h5"
# filename_weights = "./" + filename_base + "_weights.h5"
filename_output_buy = "./" + filename_base + "_output_buy.h5"
filename_output_sell = "./" + filename_base + "_output_sell.h5"

# model.save(filename_model)
# model.save_weights(filename_weights)
dfBuy.to_csv(filename_output_buy, index=False)
dfSell.to_csv(filename_output_sell, index=False)

# files.download(filename_model)
# files.download(filename_weights)
# files.download(filename_output_buy)
# files.download(filename_output_sell)